# **Import**

In [3]:
import numpy as np
import pandas as pd
import copy
from numpy import dot
import math

file_path = './'

# **Read File**

In [4]:
## list
query_list = []
doc_list = []
all_query = []
all_doc = []
query_data = [] ##split的結果
query_voc_dict = {}

##query list
with open(file_path+'query_list.txt','r') as f:
    query = f.readlines()
    for i in range(len(query)):
        query_list.append(query[i].strip('\n'))

## doc list
with open(file_path+'doc_list.txt','r') as f:
    doc = f.readlines()
    for i in range(len(doc)):
        doc_list.append(doc[i].strip('\n'))
##all query 先讀進來建voc
for i in range(len(query_list)):
    with open(file_path+'./queries/'+str(query_list[i])+'.txt','r') as f:
        q_content = f.readlines()
        all_query.append(q_content)
        q_content_split = q_content[0].split()  ##split string
        query_data.append(q_content_split)
        for j in q_content_split: ##建字典
            query_voc_dict[j] = 0


In [5]:

## 各個dict&matrix
idf_dict = query_voc_dict.copy()
query_tf_matrix = np.zeros((50,123))
doc_tf_matrix = np.zeros((4191,123))
## doc length
avg_doc_len = 0

##query的tf
for i in range(len(query_data)):
    query_dict = query_voc_dict.copy()
    for j in query_data[i]:
        if j in query_voc_dict:
            query_dict[j] += 1
    query_tf_matrix[i] = np.array(list(query_dict.values()))

## all doc idf跟tf 
for i in range(len(doc_list)):
    doc_dict = query_voc_dict.copy() ##複製一個只有 key的
    update = {} ##拿來判斷是否用過
    with open(file_path+'./docs/'+str(doc_list[i])+'.txt','r') as f:
        d_content = f.readlines()
        
        if(len(d_content)!=0): ##避免那篇有問題的
            d_content_split = d_content[0].split() ## split string
            avg_doc_len += len(d_content[0])
            for j in d_content_split:   ##all word
                if j in query_voc_dict: ## if in dict
                    doc_dict[j] += 1 ##doc的 tf
                    if (not(j in update)):
                        idf_dict[j]+=1
                        update[j] = 0 ##used
        doc_tf_matrix[i]= np.array(list(doc_dict.values())) ##得到tf值矩陣
        all_doc.append(d_content)

avg_doc_len /= len(doc_list)

In [8]:
def BM25(doc_tf_matrix,query_tf_matrix,idf_dict):
    K1 = 3.5
    K3 = 1
    b = 0.75
    N = len(all_doc) ##總共文章
    all_ans_list = []
    query_ans_list = [] ## not sort
    idf_score = np.array(list(idf_dict.values()))
    query_tf = np.zeros((50,123))
    doc_tf = np.zeros((4191,123))
    # doc_T = doc.T ##Transpose
    for i in range(doc_tf_matrix.shape[0]):
        for j in range(doc_tf_matrix.shape[1]):
            if len(all_doc[i]) != 0:
                tf_dot = doc_tf_matrix[i][j] / (1 - b + b * (len(all_doc[i][0]) / avg_doc_len))
                tf = ((K1 + 1) * tf_dot / (K1 + tf_dot))
                if tf > 0:
                    tf += 0.2
                doc_tf[i][j] = tf
    # print(doc_tf[1])
    for i in range(query_tf_matrix.shape[0]):
        for j in range(query_tf_matrix.shape[1]):
            tf = (K3 + 1) * query_tf_matrix[i][j] / (K3 + query_tf_matrix[i][j])
            query_tf_idf = tf * math.log((N - idf_score[j] + 0.5) / (idf_score[j] + 0.5)) * math.log((N - idf_score[j] + 0.5) / (idf_score[j] + 0.5)) 
            query_tf[i][j] = query_tf_idf
        result = np.matmul(doc_tf,query_tf[i])
        query_ans_list.append(result)
    for i in range(len(query_ans_list)): ##50個query
    #     ans_list = []
        ans_list = sorted(range(len(query_ans_list[i])),reverse = True ,key = lambda k : query_ans_list[i][k]) 
        result_list = []
        for j in ans_list:
            result_list.append(doc_list[j])
        all_ans_list.append(result_list)
    return all_ans_list

In [9]:
ans_list = BM25(doc_tf_matrix,query_tf_matrix,idf_dict)

save_path = './'
with open(save_path+'hw2_result_BM25.txt','w') as f:
    f.write('Query,RetrievedDocuments\n')
    for i in range(50): 
        f.write(query_list[i])
        f.write(',')
        for j in range(len(ans_list[0])):
            f.write(ans_list[i][j])
            f.write(' ')
        f.write('\n')
    f.close()